## Create and Destroy a Storage Account & Container in Azure via Terraform

- An Azure Storage Account is used to store various types of data in Azure.
  - Since an Azure Storage Account is an Azure resource, it must be placed in an Azure Resource Group.
- A Storage Container is a specific storage type taht can be created under an Azure Storage Account.
  - A Storage Container is used to store BLOBs (Binary Large OBjects), i.e. an arbitrarty file (e.g. video file).
- This Terraform Project consists of the Terraform files listed below:

In [1]:
#!dir *.tf # use this on Windows
!ls *.tf

container-registry.tf  resource-group.tf   storage-container.tf
providers.tf	       storage-account.tf  variables.tf


## Terraform providers

- We are using the same Terraform proviers as before (i.e. the `azurerm` provider for Azure).

In [2]:
#!type providers.tf # use this on Windows
!cat providers.tf

# Initialises Terraform providers and sets their version numbers.

terraform {
  required_providers {
    azurerm = {
      source  = "hashicorp/azurerm"
      version = "3.90.0"
    }
  }

  required_version = "~> 1.7.2"
}

provider "azurerm" {
  features {}
}

## Terraform variables

- We are using the same Terraform variables as before (no variable for a kubernetes cluster this time).

**Note! Make sure you change the value for the variable `app_name` to something unique!**

In [3]:
#!type variables.tf # use this on Windows
!cat variables.tf

# Sets global variables for this Terraform project.

variable "app_name" {
  default = "tsfn14g00"
}

variable "location" {
  default = "westeurope"
}

## Azure Resource Group

- We are using the same Azure Resoure Group as before.

In [4]:
#!type resource-group.tf # use this on Windows
!cat resource-group.tf

# Creates a resource group in your Azure account.

resource "azurerm_resource_group" "main" {
  name     = var.app_name
  location = var.location
}

## Azure Container Registry

- We are using the same Azure Container Registry as before.

In [5]:
#!type container-registry.tf # use this on Windows
!cat container-registry.tf

# Creates a container registry in Azure (for Docker images).

resource "azurerm_container_registry" "main" {
  name                = var.app_name
  resource_group_name = azurerm_resource_group.main.name
  location            = var.location
  admin_enabled       = true
  sku                 = "Basic"
}

## Let's view the contents of the file `storage-account.tf`

- Here we are defining an Azure Storage Account
  - The Block Type is `resource`.
  - The first Block Label is `azurerm_storage_account`
    - `azurerm` is the name of the provider (i.e. the provider for Azure defined in the file `providers.tf`).
    - `storage_account` is the name of the Azure resource (i.e. an Azure Storage Account defined in the `azurerm` provider/plugin).
  - The second Block Label is `main`, which is used to uniquely identify the resource in the Terraform files.
  - The first Argument sets the Azure Storage Account's name
    - `name` is the argument's name
    - Its value is retrieved from the Terraform variable `app_name` (defined in the file `variables.tf`).
  - The second Argument sets the Azure Resource Group in which the Azure Storage Account will be created
    - `resource_group_name` is the argument's name
    - Its value is retrieved from the Terraform Expression `azurerm_resource_group.main.name`.
      - The `azurerm_resource_group.main` Block is defined in `resource-group.tf` as `resource "azurerm_resource_group" "main"`.
      - In this Block, there is an Argument with a name of `name` who's value is defined as `var.app_name`.
      - This is the value that is assigned to `resource_group_name`.
  - The third Argument sets the Azure Storage Account's Location
    - `location` is the argument's name
    - Its value is retrieved from the Terraform variable `location` (defined in the file `variables.tf`).
  - The fourth Argument sets the Storage Account's type of storage
    - `account_kind` is the argument's name
    - Its value is set to `"StorageV2"` (which is what we want if we want to add a Storage Container to the Storage Account).
    - For more information about the Azure Storage Account's account type, see:
      - https://registry.terraform.io/providers/hashicorp/azurerm/latest/docs/resources/storage_account
      - https://learn.microsoft.com/en-us/azure/storage/common/storage-account-overview
  - The fifth Argument sets the Storage Account's tier
    - `account_tier` is the argument's name
    - Its value is set to `"Standard"` (i.e. the Standard Tier which is a cheaper alternative).
    - For more information about the Azure Storage Account's tier, see:
      - https://registry.terraform.io/providers/hashicorp/azurerm/latest/docs/resources/storage_account
      - https://learn.microsoft.com/en-us/azure/storage/common/storage-account-overview
  - The fifth Argument sets the Storage Account's replication type.
    - `account_replication_type` is the argument's name
    - Its value is set to `"LRS"` (for Local Redundant Storage).
    - For more information about the Azure Storage Account's replication type, see:
      - https://registry.terraform.io/providers/hashicorp/azurerm/latest/docs/resources/storage_account
      - https://learn.microsoft.com/en-us/azure/storage/common/storage-account-overview

In [6]:
#!type storage-account.tf # use this on Windows
!cat storage-account.tf

# Create a storage account in Azure

# Note!
# - Resource "azurerm_resource_group.main" with a property "name" is defined in the file "resource-group.tf".
# - The value for "resource_group_name" below is set using property "name" in resource "azurerm_resource_group.main":
#   - resource_group_name = azurerm_resource_group.main.name
# - "name" and "location" below are set from Terraform variables defined in the file "variables.tf".
# - The value for "storage_account_name" below is set using property "name" in resource "azurerm_storage_account.main":
#   - storage_account_name = azurerm_storage_account.main.name

resource "azurerm_storage_account" "main" {
  name                     = var.app_name
  resource_group_name      = azurerm_resource_group.main.name
  location                 = var.location
  account_kind             = "StorageV2"
  account_tier             = "Standard"
  account_replication_type = "LRS"
}

## Let's view the contents of the file `storage-container.tf`

- Here we are defining an Azure Storage Container (which is used to store data as BLOBs, i.e. arbitrary files)
  - The Block Type is `resource`.
  - The first Block Label is `azurerm_storage_container`
    - `azurerm` is the name of the provider (i.e. the provider for Azure defined in the file `providers.tf`).
    - `storage_container` is the name of the Azure resource (i.e. an Azure Storage Container defined in the `azurerm` provider/plugin).
  - The second Block Label is `main`, which is used to uniquely identify the resouce in the Terraform files.
  - The first Argument sets the Storage Container's name
    - `name` is the argument's name
    - Its value is set to `videos` (this is equivalent to the name of a folder, which will contain files).
  - The second Argument sets the Azure Storage Account in which the Storage Container will be created
    - `storage_account_name` is the argument's name
    - Its value is retrieved from the Terraform Expression `azurerm_storage_account.main.name`.
      - The `azurerm_storage_account.main` Block is defined in `storage-account.tf`
        - as `resource "azurerm_storage_account" "main"`.
      - In this Block, there is an Argument with a name of `name` who's value is defined as `var.app_name`.
      - This is the value that is assigned to `storage_account_name`.
  - The third Argument sets the Storage Container's access type
    - `container_access_type` is the argument's name
    - Its value is set to `private` (which means public access to the Storage Container is disabled).
    - For more information about the Azure Storage Container's acces type, see:
      - https://registry.terraform.io/providers/hashicorp/azurerm/latest/docs/resources/storage_container
      - https://learn.microsoft.com/en-us/azure/private-link/tutorial-private-endpoint-storage-portal?tabs=dynamic-ip

In [7]:
#!type storage-container.tf # use this on Windows
!cat storage-container.tf

# Create a storage container in Azure (for BLoBs, i.e. arbitrary files)

# Note!
# - Resource "azurerm_storage_account.main" with a property "name" is defined in the file "storage-account.tf".
# - The value for "storage_account_name" below is set using property "name" in resource "azurerm_storage_account.main":
#   - storage_account_name  = azurerm_storage_account.main.name

resource "azurerm_storage_container" "main" {
  name                  = "videos"
  storage_account_name  = azurerm_storage_account.main.name
  container_access_type = "private"
}

## Initialize Terraform

- We initialize the Terraform Project as before.

In [8]:
#rm -rf .terraform rm .terraform.lock.hcl terraform.tfstate terraform.tfstate.backup
!terraform init


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/azurerm versions matching "3.90.0"...
- Installing hashicorp/azurerm v3.90.0...
- Installed hashicorp/azurerm v3.90.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


## Terraform Apply

- We apply the Terraform Project as before.

In [9]:
!terraform apply -auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_container_registry.main will be created
  + resource "azurerm_container_registry" "main" {
      + admin_enabled                 = true
      + admin_password                = (sensitive value)
      + admin_username                = (known after apply)
      + encryption                    = (known after apply)
      + export_policy_enabled         = true
      + id                            = (known after apply)
      + location                      = "westeurope"
      + login_server                  = (known after apply)
      + name                          = "tsfn14g00"
      + network_rule_bypass_option    = "AzureServices"
      + network_rule_set              = (known after apply)
      + public_network_access_enabled = true
      + resource_group_name           = "tsf

## List Azure Resource Groups

- The Azure CLI command `az group list -o table` lists all Resource Groups in Azure.
- We see that the Resource Group defined in the Terraform project has been created.

In [10]:
!az group list -o table

Name       Location    Status
---------  ----------  ---------
tsfn14g00  westeurope  Succeeded


## List Resources in Resource Group `tsfn14g00`

- The Azure CLI command `az resource list -n tsfn14g00 -o table` lists resources in Resource Group `tsfn14g00`.
- We see that the Resource Group contains the Storage Account and the Container Registry.

In [11]:
!az resource list -n tsfn14g00 -o table

Name       ResourceGroup    Location    Type                                    Status
---------  ---------------  ----------  --------------------------------------  --------
tsfn14g00  tsfn14g00        westeurope  Microsoft.ContainerRegistry/registries
tsfn14g00  tsfn14g00        westeurope  Microsoft.Storage/storageAccounts


## List Azure Storage Accounts

- The Azure CLI command `az storage account list -o table` lists all Storage Accounts in Azure.
- We see that the Storage Account defined in the Terraform project has been cerated.

In [12]:
#!az storage account list --resource-group $RESOURCE_GROUP -o table
!az storage account list -o table

AccessTier    AllowBlobPublicAccess    AllowCrossTenantReplication    AllowSharedKeyAccess    CreationTime                      DefaultToOAuthAuthentication    EnableHttpsTrafficOnly    EnableNfsV3    IsHnsEnabled    IsSftpEnabled    Kind       Location    MinimumTlsVersion    Name       PrimaryLocation    ProvisioningState    PublicNetworkAccess    ResourceGroup    StatusOfPrimary
------------  -----------------------  -----------------------------  ----------------------  --------------------------------  ------------------------------  ------------------------  -------------  --------------  ---------------  ---------  ----------  -------------------  ---------  -----------------  -------------------  ---------------------  ---------------  -----------------
Hot           True                     True                           True                    2024-02-10T17:24:12.042778+00:00  False                           True                      False          False           False      

## List Storage Account Access Keys for Storage Account `tsfn14g00`

- The Azure CLI command `az storage account keys list --account-name tsfn14g00 -o table` will
  - List Access Keys for the Storage Account named `tsfn14g00`.
- It shows the two keys `key1` and `key2` of which any can be used together with the Storage Account `name` to acccess the Storage Account.
  - **This is the KEY (key1 or key2) you would use together with the storage account NAME to upload/download/delete files.**

In [13]:
#!az storage account keys list --account-name tsfn14g00 --resource-group tsfn14g00 -o table
!az storage account keys list --account-name tsfn14g00 -o table

# Let's store the STORAGE ACCOUNT NAME and STORAGE ACCES KEY in Python variables so we can use them later in this notebook
STORAGE_ACCOUNT_NAME=!az storage account list --query [0].name -o tsv
STORAGE_ACCOUNT_NAME=STORAGE_ACCOUNT_NAME[0]
STORAGE_ACCESS_KEY=!az storage account keys list --account-name tsfn14g00 --resource-group tsfn14g00 --query [0].value -o tsv
STORAGE_ACCESS_KEY=STORAGE_ACCESS_KEY[0]

CreationTime                      KeyName    Permissions    Value
--------------------------------  ---------  -------------  ----------------------------------------------------------------------------------------
2024-02-10T17:24:12.198994+00:00  key1       FULL           cSCfacjAWkSdgweG0pNOvj3G/xCFa9IDY+2Vs1VIYTOBD+KMImk0f0F7GD7J9JXZWUXMkaGL1C1P+AStExxj5A==
2024-02-10T17:24:12.198994+00:00  key2       FULL           /qZtPfHBQt5WUQG5VccLw9gpnc5GatYMqa0qhJXBhOn77219Qp/5o35caQA7Gm798a1slqtzZYJ1+ASt+uwT5w==


## List Containers in Storage Account `tsfn14g00`

- The command `az storage container list --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table`
  - Lists the Storage Containers in Storage Account `$STORAGE_ACCOUNT_NAME` (using the Key `$STORAGE_ACCESS_KEY`).

In [14]:
!az storage container list --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Name    Lease Status    Last Modified
------  --------------  -------------------------
videos                  2024-02-10T17:24:33+00:00


## Create Container `videos2` in Storage Container `tsfn14g00`

- The command:

  ```bash
  az storage container create --name videos2 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table
  ```
  - Creates a new Storage COntainer named `videos2` in Storage Account `$STORAGE_ACCOUNT_NAME` (using KEY `$STORAGE_ACCESS_KEY`).

In [15]:
!az storage container create --name videos2 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Created
---------
True


## List Containers in Storage Container `tsfn14g00`

- Now when we list the containes in Storage Account `tsfn14g00` again, we see there are two Storage Containers.

In [16]:
!az storage container list --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Name     Lease Status    Last Modified
-------  --------------  -------------------------
videos                   2024-02-10T17:24:33+00:00
videos2                  2024-02-10T17:25:16+00:00


## Delete Container `videos2` in Storage Container `tsfn14g00`

- The command:

  ```bash
  az storage container delete --name videos2 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table
  ```
  - Deletes the Storage COntainer named `videos2` from Storage Account `$STORAGE_ACCOUNT_NAME` (using KEY `$STORAGE_ACCESS_KEY`).

In [17]:
!az storage container delete --name videos2 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Deleted
---------
True


## List Containers in Storage Container `tsfn14g00`

- Now when we list the containes in Storage Account `tsfn14g00` again, we see the Storage Container has been deleted.

In [18]:
!az storage container list --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Name    Lease Status    Last Modified
------  --------------  -------------------------
videos                  2024-02-10T17:24:33+00:00


## List Blobs (Files) in Container `videos` in Storage Container `tsfn14g00`

- The command:

  ```bash
  az storage blob list --container-name videos --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY
  ```
  - Lists files stored in the Storage Container named `videos` in Storage Account `$STORAGE_ACCOUNT_NAME` (using KEY `$STORAGE_ACCESS_KEY`).
  - Currently the lsit is empty, since no files are stored in Storage Container `videos`.

In [19]:
!az storage blob list --container-name videos --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table 

## List Local Video File

- There is a video file stored in the host machine's file system: `video-storage/videos/SampleVideo_1280x720_1mb.mp4`.

In [20]:
#!dir video-storage/videos # use this on Widnows
!ls video-storage/videos

SampleVideo_1280x720_1mb.mp4


## Upload a file to Container `videos` in Storage Container `tsfn14g00`

- The command below:
  - Uploads the local file `video-storage/videos/SampleVideo_1280x720_1mb.mp4` (`--file`).
  - To the Storage Container `videos` (`--container-name`).
  - In Storage Account `$STORAGE_ACCOUNT_NAME` (`--account-name`).
  - using Key `$STORAGE_ACCESS_KEY` (`--account-key`).

In [21]:
!az storage blob upload --container-name videos --name SampleVideo_1280x720_1mb.mp4 --file video-storage/videos/SampleVideo_1280x720_1mb.mp4 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Finished[#############################################################]  100.0000%
Client_request_id                     Content_md5               Date                       LastModified               Request_id                            Request_server_encrypted    Version
------------------------------------  ------------------------  -------------------------  -------------------------  ------------------------------------  --------------------------  ----------
69f2313c-c839-11ee-a756-256937b67ec7  1Vvd+NYpEIee2fYFUiFJqA==  2024-02-10T17:25:42+00:00  2024-02-10T17:25:42+00:00  eac52aa8-401e-0043-3046-5c9b96000000  True                        2022-11-02


## List Blobs (Files) in Container `videos` in Storage Container `tsfn14g00`

- Now if we list the files in Storage Container `videos` again, we see the video file has been uploaded.

In [22]:
!az storage blob list --container-name videos --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table 

Name                          Blob Type    Blob Tier    Length    Content Type    Last Modified              Snapshot
----------------------------  -----------  -----------  --------  --------------  -------------------------  ----------
SampleVideo_1280x720_1mb.mp4  BlockBlob    Hot          1055736   video/mp4       2024-02-10T17:25:42+00:00


## Delete Local Video File

- Let's delete the video fil from the host machine.

In [23]:
#!del video-storage/videos/SampleVideo_1280x720_1mb.mp4 # use this on Windows
#!dir video-storage/videos # use this on Windows

!rm video-storage/videos/SampleVideo_1280x720_1mb.mp4
!ls video-storage/videos

## Download a file from Container `videos` in Storage Account `tsfn14g00`

- The command below:
  - Dowloads the file `SampleVideo_1280x720_1mb.mp4` (`--name`).
  - From Storage Container `videos` (`--container-name`).
  - In Storage Account `$STORAGE_ACCOUNT_NAME` (`--account-name`).
  - To the local file `video-storage/videos/SampleVideo_1280x720_1mb.mp4` (`--file`).
  - using Key `$STORAGE_ACCESS_KEY` (`--account-key`).

In [24]:
!az storage blob download --container-name videos --name SampleVideo_1280x720_1mb.mp4 --file video-storage/videos/SampleVideo_1280x720_1mb.mp4 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Finished[#############################################################]  100.0000%
Name                          Blob Type    Blob Tier    Length    Content Type    Last Modified              Snapshot
----------------------------  -----------  -----------  --------  --------------  -------------------------  ----------
SampleVideo_1280x720_1mb.mp4  BlockBlob                 1055736   video/mp4       2024-02-10T17:25:42+00:00


## List Local Video File

- We see that the file has been downlaoded to the host machine.

In [25]:
#!dir video-storage/videos # use this on Windows
!ls video-storage/videos

SampleVideo_1280x720_1mb.mp4


## Delete a file from Container `videos` in Storage Container `tsfn14g00`

- The command below:
  - Deletes the file `SampleVideo_1280x720_1mb.mp4` (`--name`).
  - From Storage Container `videos` (`--container-name`).
  - In Storage Account `$STORAGE_ACCOUNT_NAME` (`--account-name`).
  - using Key `$STORAGE_ACCESS_KEY` (`--account-key`).

In [26]:
!az storage blob delete --container-name videos --name SampleVideo_1280x720_1mb.mp4 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

## List Blobs (Files) in Container `videos` in Storage Container `tsfn14g00`

- Now if we list the files in Storage Container `videos` again, we see the video file has been deleted.

In [27]:
!az storage blob list --container-name videos --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table 

## Upload file again to Container `videos` in Storage Container `tsfn14g00`

- Let's upload the video file to the Storage Container again. 

In [28]:
!az storage blob upload --container-name videos --name SampleVideo_1280x720_1mb.mp4 --file video-storage/videos/SampleVideo_1280x720_1mb.mp4 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table
!az storage blob list --container-name videos --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table 

Finished[#############################################################]  100.0000%
Client_request_id                     Content_md5               Date                       LastModified               Request_id                            Request_server_encrypted    Version
------------------------------------  ------------------------  -------------------------  -------------------------  ------------------------------------  --------------------------  ----------
7b25f268-c839-11ee-a756-256937b67ec7  1Vvd+NYpEIee2fYFUiFJqA==  2024-02-10T17:26:11+00:00  2024-02-10T17:26:11+00:00  b545a92e-d01e-007e-4346-5ced8d000000  True                        2022-11-02
Name                          Blob Type    Blob Tier    Length    Content Type    Last Modified              Snapshot
----------------------------  -----------  -----------  --------  --------------  -------------------------  ----------
SampleVideo_1280x720_1mb.mp4  BlockBlob    Hot          1055736   video/mp4       2024-02-10T17:2

## List Azure Container Registries

- The Azure CLI command `az acr list -o table` lists all Container Registries in Azure.
- We see that the Container Registry defined in the Terraform project has been created.

In [29]:
!az acr list -o table

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-10T17:24:12Z  True


## List Repositories in Container Registry `tsfn14g00 `

- The Azure CLI command `az acr repository list -n tsfn14g00 --top 10 -o table` lists Repositories in a Azure Container Registry `tsfn14g00`.
  - The `-n` option is manditory and specifies the `NAME` of the Container Registry.
  - The `--top 10` limits the list to the first 10 Repositories (remove to see all Repositories).
- We see that Container Registry `tsfn14g00` doesn't contain any Repositories.

In [30]:
!az acr repository list -n tsfn14g00 --top 10 -o table

## Show Information about Azure Container Registry `tsfn14g00`

- The Azure CLI command `az acr show -n tsfn14g00 -o table` shows information about Container Registry `tsfn14g00`.
- It shows the Container Registry's `LOGIN SERVER` which is the URL to your Container Registry on Azure.
  - **This is the URL you would use to upload Docker Images to the Azure Container Registry.**

In [31]:
!az acr show -n tsfn14g00 -o table

# Let's store the LOGIN SERVER in a Python variable so we can use it later in this notebook
CONTAINER_REGISTRY_LOGIN_SERVER=!az acr show -n tsfn14g00 --query loginServer -o tsv
CONTAINER_REGISTRY_LOGIN_SERVER=CONTAINER_REGISTRY_LOGIN_SERVER[0]

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-10T17:24:12Z  True


## Show Credentials for Azure Container Registry `tsfn14g00`

- The Azure CLI command `az acr credential show -n tsfn14g00 -o table` shows credentials about Container Registry `tsfn14g00`.
- It shows the Container Registry's `USERNAME` and  `PASSWORD` to use to authenticate with your Azure Container Registry.
  - **This is the USERNAME and PASSWORD you would use to login to Docker to upload Images to the Azure Container Registry.**

In [32]:
!az acr credential show -n tsfn14g00 -o table

# Let's store the USERNAME and PASSWORD in Python variables so we can use them later in this notebook
CONTAINER_REGISTRY_USERNAME=!az acr credential show -n tsfn14g00 --query username -o tsv
CONTAINER_REGISTRY_USERNAME=CONTAINER_REGISTRY_USERNAME[0]
CONTAINER_REGISTRY_PASSWORD=!az acr credential show -n tsfn14g00 --query passwords[0].value -o tsv
CONTAINER_REGISTRY_PASSWORD=CONTAINER_REGISTRY_PASSWORD[0]

USERNAME    PASSWORD                                              PASSWORD2
----------  ----------------------------------------------------  ----------------------------------------------------
tsfn14g00   A3Ytz36bV0+JRpX5Lss38bbQTqmIEuk11+AYAhbHl6+ACRCsrNDp  oNc2EjP/cCScm6ZejaawKqld70VEiKg4DUTyxU2aO3+ACRD45Uyg


## Login to Azure Container Registry via Docker

- Replace the variables below with your `LOGIN_SERVER`, `USERNAME` and `PASSWORD`.

In [34]:
!docker login $CONTAINER_REGISTRY_LOGIN_SERVER -u $CONTAINER_REGISTRY_USERNAME -p $CONTAINER_REGISTRY_PASSWORD

# In Ubuntu with environment variables CONTAINER_REGISTRY_LOGIN_SERVER, CONTAINER_REGISTRY_USERNAME and CONTAINER_REGISTRY_USERNAME
#!echo $CONTAINER_REGISTRY_PASSWORD | docker login $CONTAINER_REGISTRY_LOGIN_SERVER -u $CONTAINER_REGISTRY_USERNAME --password-stdin  > /dev/null 2>&1

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded


## Let's view the code in `video-storage/src/index.js`

- The file contains code for creating a simple Node.js application, including the steps:
  1. Require the NPM package "express".
  2. Require the classes BlobServiceClient and StorageSharedKeyCredential from NPM packge "@azure/storage-blob".
  3. Read the environment variable PORT to use as the Express web server's listen port.
  4. Read the environment variable STORAGE_ACCOUNT_NAME to login to the Azure Storage Account.
  5. Read the environment variable STORAGE_ACCESS_KEY to login to the Azure Storage Account.
  6. Create a function createBlobServer() that creates a BlobServiceClient use to comminucate with the Storage Account.
  7. Create an instance of an Express web server.
  8. Register a GET route for the "/video" path.
     1. Get the "path" from the request's query string (which contains the name of the file in Azure).
     2. Create a BlobService instance (which also logs in to the Storage Account).
     3. Create a ContainerClient for the "videos" Storage Container.
     4. Create a BlobClient for the video file in the Storage Container.
     5. Get the file properties via the BlobClient.
     6. Write the HTTP response header including information about the file size in bytes and content type (video/mp4).
     7. Stream the file contents from the Storage Container to the HTTP GET response.
  9. Start the Express web server listening on the port stored in environment variable PORT.
- As we see, this code produces a simple microservice that streams a video file from the Azure Storage Container to an HTTP GET response.

In [35]:
cat video-storage/src/index.js

const express = require("express");
const { BlobServiceClient, StorageSharedKeyCredential } = require("@azure/storage-blob");

//
// Throws an error if the any required environment variables are missing.
//

if (!process.env.PORT) {
    throw new Error("Please specify the port number for the HTTP server with the environment variable PORT.");
}

if (!process.env.STORAGE_ACCOUNT_NAME) {
    throw new Error("Please specify the name of an Azure storage account in environment variable STORAGE_ACCOUNT_NAME.");
}

if (!process.env.STORAGE_ACCESS_KEY) {
    throw new Error("Please specify the access key to an Azure storage account in environment variable STORAGE_ACCESS_KEY.");
}

//
// Extracts environment variables to globals for convenience.
//

const PORT = process.env.PORT;
const STORAGE_ACCOUNT_NAME = process.env.STORAGE_ACCOUNT_NAME;
const STORAGE_ACCESS_KEY = process.env.STORAGE_ACCESS_KEY;

console.log(`Serving videos from Azure storage account ${STORAGE_ACCOUNT_NAME}.`);

//
// Create

## Initialize the Node.js project

- Let's initialise the Node.js project for the microservice by:
  - Running `npm init`.
  - Running `npm pkg` to configure `package.json`.
  - Running `npm install` to install the required NPM packages.

In [36]:
%%system
#rm -rf ./video-storage/node_modules ./video-storage/package.json ./video-storage/package-lock.json
cd video-storage
npm init -y
npm pkg set 'main'='./src/index.js'
npm pkg set 'scripts.start'='node ./src/index.js'
npm pkg set 'scripts.start:dev'='nodemon ./src/index.js'
npm pkg delete 'scripts.test'
npm install --silent --save express@5.0.0-beta.1
npm install --silent --save @azure/storage-blob@12.17.0
npm install --silent --save-dev nodemon@3.0.3
cd ..

['Wrote to /home/patrick/projects/tsfn14/tsfn14/03_Azure_and_Terraform/05_storage_account/video-storage/package.json:',
 '',
 '{',
 '  "name": "video-storage",',
 '  "version": "1.0.0",',
 '  "description": "",',
 '  "main": "index.js",',
 '  "scripts": {',
 '    "test": "echo \\"Error: no test specified\\" && exit 1"',
 '  },',
 '  "keywords": [],',
 '  "author": "",',
 '  "license": "ISC"',
 '}',
 '',
 '']

## Let's look at the Docker file used to build an image of the Node.js project

- Our image is based on the slimmed-down `alpine` version of `node:19.9.0`.
- The working directory in the image is `/usr/src/app`.
- The files `package.json` and `package-lock.json` are copied from the host computer to the image's working directory.
- The command `npm ci --omit-dev` is run in the image's working directory which installs non-dev NPM packages from `package.json`.
- The file `index.js` is copied from the host computer's `src` folder to the image's working directory's `src` folder.
- The command `npm start` will be run in the working directory when an image container is started.

In [37]:
#!type video-storage/Dockerfile # use this on Windows
!cat video-storage/Dockerfile

FROM node:19.9.0-alpine

WORKDIR /usr/src/app
COPY package*.json ./
RUN npm ci --omit=dev
COPY ./src ./src

CMD npm start

## Build and Push a Docker Image to Azure Container Registry

- Here we are building an image of the Node.js application (microservice).
- We are tagging the image as `tsfn14g00.azurecr.io/video-storage:1,`where:
  - `tsfn14g00.azurecr.io` is the URL (LOGIN SERVER) to our Container Registry.
  - `video-storage` is the name of our image (repository).
  - `1` is the version of the image (tag).
- Then the image is pushed to the Azure Container Registry.
- Finally, the local image is removed from the host computer.

In [38]:
%%system

# Build Docker image with Nodejs Application
docker build -q -t {CONTAINER_REGISTRY_LOGIN_SERVER}/video-storage:1 -f ./video-storage/Dockerfile ./video-storage

# Push Docker Image to Azure Container Registry
docker push {CONTAINER_REGISTRY_LOGIN_SERVER}/video-storage:1

# Clean up
docker rmi {CONTAINER_REGISTRY_LOGIN_SERVER}/video-storage:1
docker images {CONTAINER_REGISTRY_LOGIN_SERVER}/video-storage:1

['failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory',
 '',
 'DEPRECATED: The legacy builder is deprecated and will be removed in a future release.',
 '            Install the buildx component to build images with BuildKit:',
 '            https://docs.docker.com/go/buildx/',
 '',
 'sha256:668d7515881255b99578196e3940ad1e842107fde64c19e305d008caf18af0bc',
 'The push refers to repository [tsfn14g00.azurecr.io/video-storage]',
 'b320483b5121: Preparing',
 '56d5ff0e8727: Preparing',
 'ddc9fad26382: Preparing',
 '68c1df7ad2a4: Preparing',
 'e394b5a78515: Preparing',
 '9b2d5af7f709: Preparing',
 'b240fe81adaa: Preparing',
 'bb01bd7e32b5: Preparing',
 'b240fe81adaa: Waiting',
 'bb01bd7e32b5: Waiting',
 '9b2d5af7f709: Waiting',
 'b320483b5121: Pushed',
 '68c1df7ad2a4: Pushed',
 '56d5ff0e8727: Pushed',
 'ddc9fad26382: Pushed',
 'e394b5a78515: Pushed',
 '9b2d5af7f709: Pushed',
 'bb01bd7e32b5: Pushed',
 'b240fe81adaa: Pushed',
 '1: dige

## List Repositories in Container Registry `tsfn14g00 `

- We see that the Repository `video-storage` has been created in Container Registry `tsfn14g00`.

In [39]:
!az acr repository list -n tsfn14g00 --top 10 -o table

Result
-------------
video-storage


## Show Information about Repository `video-storage`

- The Azure CLI command `az acr repository show -n tsfn14g00 --repository video-storage -o table`
  - Shows information about Repository `video-storage` in Container Registry `tsfn14g00`.
    - It contains images named `video-storage` (ImageName).
    - It has a tag count of `1` (TagCount), i.e. currently there is only one tag for the `video-storage` image.

In [40]:
!az acr repository show -n tsfn14g00 --repository video-storage -o table

CreatedTime                   ImageName      LastUpdateTime                ManifestCount    Registry              TagCount
----------------------------  -------------  ----------------------------  ---------------  --------------------  ----------
2024-02-10T17:28:07.1753421Z  video-storage  2024-02-10T17:28:07.2573167Z  1                tsfn14g00.azurecr.io  1


## List Tags in Repository `video-storage`

- The Azure CLI command `az acr repository show-tags -n tsfn14g00 --repository video-storage --top 10 -o table`:
  - Lists the tags in Repository `video-storage` in Container Registry `tsfn14g00`.
    - Currently there is only one tag.
    - The tag has the value `1`.

In [41]:
!az acr repository show-tags -n tsfn14g00 --repository video-storage --top 10 -o table

Result
--------
1


## Show Information about Image `video-streaming:1`

- The Azure CLI command `az acr repository show -n tsfn14g00 --image video-storage:1 -o table`:
  - Shows information about image `video-storage:1` in Container Registry `tsfn14g00`.
    - The information includes the Digest for the image.

In [42]:
!az acr repository show -n tsfn14g00 --image video-storage:1 -o table

CreatedTime                   Digest                                                                   LastUpdateTime                Name    Signed
----------------------------  -----------------------------------------------------------------------  ----------------------------  ------  --------
2024-02-10T17:28:07.2867885Z  sha256:a743d6d948313026694b3289a2e3974ea5329b9c04c3acc772f207e18601c2e1  2024-02-10T17:28:07.2867885Z  1       False


## Show Azure Container Registry Usage

- The command `az acr show-usage -n tsfn14g00 -o table` shows the usage of Container Registry `tsfn14g00`.
  - We can see the maximum number of allowed bytes in the `LIMIT` column (first row).
  - We can see the current number of bytes in the `CURRENT VALUE` column (first row).

In [43]:
!az acr show-usage -n tsfn14g00 -o table

NAME       LIMIT        CURRENT VALUE    UNIT
---------  -----------  ---------------  ------
Size       10737418240  60578333         Bytes
Webhooks   2            0                Count
ScopeMaps  100          0                Count
Tokens     100          0                Count


## Pull and Run an Image from the Azure Container Registry

- The image `video-storage:1` will be pulled from the Azure Container Registry to the host computer.
- Then a Container is created from the Image, where:
  - The host computer's port 3000 is mapped to the container's port 3000.
  - An environment variable PORT is created in the container with the value 3000.
  - An environment variable STORAGE_ACCOUNT_NAME is created in the container with the Storage Account's Name.
  - An environment variable STORAGE_ACCESS_KEY is created in the container with the Storage Access Key.

In [44]:
%%system

docker run --name video-storage -d -p 3000:3000 -e PORT=3000 -e STORAGE_ACCOUNT_NAME={STORAGE_ACCOUNT_NAME} -e STORAGE_ACCESS_KEY={STORAGE_ACCESS_KEY} {CONTAINER_REGISTRY_LOGIN_SERVER}/video-storage:1
docker ps
docker logs video-storage
#docker exec -it video-storage /bin/sh

["Unable to find image 'tsfn14g00.azurecr.io/video-storage:1' locally",
 '1: Pulling from video-storage',
 '8a49fdb3b6a5: Already exists',
 '1197750296b3: Already exists',
 'f352bc07f19b: Already exists',
 '47be83a79857: Already exists',
 '5e5985408991: Pulling fs layer',
 '44104ea2832f: Pulling fs layer',
 '0efd076f6ac1: Pulling fs layer',
 '164665fd9637: Pulling fs layer',
 '164665fd9637: Waiting',
 '44104ea2832f: Verifying Checksum',
 '5e5985408991: Download complete',
 '5e5985408991: Pull complete',
 '44104ea2832f: Pull complete',
 '164665fd9637: Verifying Checksum',
 '164665fd9637: Download complete',
 '0efd076f6ac1: Verifying Checksum',
 '0efd076f6ac1: Download complete',
 '0efd076f6ac1: Pull complete',
 '164665fd9637: Pull complete',
 'Digest: sha256:a743d6d948313026694b3289a2e3974ea5329b9c04c3acc772f207e18601c2e1',
 'Status: Downloaded newer image for tsfn14g00.azurecr.io/video-storage:1',
 '4e88d932fb0133d91eb1aeca26865db117a276b4405d2d754cb0d5ba9e29f191',
 'CONTAINER ID   IMA

## Access the Microservice from a Web Browser

- Open a web browser and enter the URL `http://localhost:3000/video?path=SampleVideo_1280x720_1mb.mp4`
  - This will send an HTTP GET request to the microservice's GET route for the "/video" path.
    - The query string `path=SampleVideo_1280x720_1mb.mp4` will also be accesible by the GET route.
  - The video stored in the storage account's storage container will be streamed to the web browser by the microservice.

In [45]:
#!firefox http://localhost:3000/video?path=SampleVideo_1280x720_1mb.mp4

## Stop and Remove the Docker Container and Image from your computer

- The Container `video-storage` is stopped and removed from the host computer.
- Then the Image `video-storage:1` is removed from the host computer.

In [46]:
%%system

docker stop video-storage
docker rm video-storage
docker ps -a
docker rmi {CONTAINER_REGISTRY_LOGIN_SERVER}/video-storage:1
docker images {CONTAINER_REGISTRY_LOGIN_SERVER}/video-storage:1

['video-storage',
 'video-storage',
 'CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES',
 'Untagged: tsfn14g00.azurecr.io/video-storage:1',
 'Untagged: tsfn14g00.azurecr.io/video-storage@sha256:a743d6d948313026694b3289a2e3974ea5329b9c04c3acc772f207e18601c2e1',
 'Deleted: sha256:668d7515881255b99578196e3940ad1e842107fde64c19e305d008caf18af0bc',
 'Deleted: sha256:638d35339eedd0e9c94569dc22fa7fad45c4f3b82e4e65bad2f39c7054fb51f2',
 'Deleted: sha256:55d175abcbca9264982f1ab4d6388863636e794acdf6a99e182b4ad9fcb7e130',
 'Deleted: sha256:768b8aad81b09b6808f98fa2a53d923c83658383dacd41aeb7b5c2177b53b450',
 'Deleted: sha256:be8ae31e3194b8be445211c18195ea4bd54ab32bca70bdcea2213881d0183120',
 'REPOSITORY   TAG       IMAGE ID   CREATED   SIZE']

## Logout from the Azure Container Registry via Docker

In [48]:
!docker logout $CONTAINER_REGISTRY_LOGIN_SERVER

Removing login credentials for tsfn14g00.azurecr.io


## Delete Repository `video-storage` from Azure Container Registry

- The Azure CLI command `az acr repository delete -y -n tsfn14g00 --repository video-storage -o table`
  - Deletes the Repository `video-storage` in Container Registry `tsfn14g00`.
  - Use this command to delete all images in Repository `video-storage`.
- The Azure CLI command `az acr repository delete -y -n tsfn14g00 --image video-storage:1 -table`
  - Deletes the Image `video-storage:1` in Container Registry `tsfn14g00`.
  - Use this command to delete one image from Repository `video-storage`.

In [49]:
#!az acr repository delete -y -n tsfn14g00 --image video-storage:1 -table
!az acr repository delete -y -n tsfn14g00 --repository video-storage -o table

## List Repositories in Container Registry `tsfn14g00 `

- We see that the Repository `video-storage` has been deleted from Container Registry `tsfn14g00`.

In [50]:
!az acr repository list -n tsfn14g00 --top 10 -o table

## Terraform Destroy

- We destroy the resources defined in the Terraform project as before.

In [51]:
!terraform destroy -auto-approve

azurerm_resource_group.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]
azurerm_container_registry.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.ContainerRegistry/registries/tsfn14g00]
azurerm_storage_account.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.Storage/storageAccounts/tsfn14g00]
azurerm_storage_container.main: Refreshing state... [id=https://tsfn14g00.blob.core.windows.net/videos]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # azurerm_container_registry.main will be destroyed
  - resource "azurerm_container_registry" "main" {
      - admin_enabled                 = true -> null
      - admin_password     

## List Azure Container Registries

- The Azure CLI command `az acr list -o table` lists all Container Registries in Azure.
- We see that the Container Registry defined in the Terraform project has been destroyed.

In [52]:
!az acr list -o table

## List Azure Storage Accounts

- The Azure CLI command `az storage account list -o table` lists all Storage Accounts in Azure.
- We see that the Storage Account defined in the Terraform project has been destroyed.

In [53]:
#!az storage account list --resource-group $RESOURCE_GROUP -o table
!az storage account list -o table

## List Azure Resource Groups

- The Azure CLI command `az group list -o table` lists all Resource Groups in Azure.
- We see that the Resource Group defined in the Terraform project has been destroyed.

In [54]:
!az group list -o table